# Why is university tuition so high in the US?

There have been lots of news stories covering how university tuition is rising in the United States and is outpacing the financial means of students (before and after graduation). With the avaiable data from the US College Scorecard dataset, I will dive into this question and try to ascertain what is largest reason for rising tuition and produce cost-effective recommendations to alleviate the issue without harming the quality of education.

## College Scorecard

It's no secret that US university students often graduate with debt repayment obligations that far outstrip their employment and income prospects. While it's understood that students from elite colleges tend to earn more than graduates from less prestigious universities, the finer relationships between future income and university attendance are quite murky. In an effort to make educational investments less speculative, the US Department of Education has matched information from the student financial aid system with federal tax returns to create the College Scorecard dataset.

For my purposes, I'm going to examine the relationship between school tuition and other features of the dataset, along with changes in tuition over the lifetime of the Scorecard.

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

## Reading data from file folder

Since there's datasets since the inception of the scorecard (1996) that could to be read in, I created a function that goes into the data folder, reads each csv, and concatenates each one onto a dataframe. Since each year of data contains over 2000 columns, it will be best if I select only the past five years to reduce runtime. It's possible that I could lose some insights by doing this. It would also be better to transfer these csv's into something like a sqlite database, but for my purposes reading and filtering with pandas should do for now. 

In [8]:
# outputs all csv's into one file and saves it as new csv 
def read_files(in_path, out_path):
    file_list = os.listdir(in_path)
    output = []
    
    for file_name in file_list:
        df = pd.read_csv(os.path.join(in_path, file_name))
        
        # adding a date column as the filename of the csv that the data point originated from
        df['DATE'] = file_name
        output.append(df)
    
    df_combined = pd.concat(output)
    df_combined.to_csv(os.path.join(out_path, 'college_scorecard_fiveyears.csv'), index = False)

In [9]:
# executing function to reorganize last five years of data
read_files('D:/Python/validere_takehome/CollegeScorecard_Raw_Data/data', 
           'D:/Python/validere_takehome/CollegeScorecard_Raw_Data/combined_data')

In [10]:
# reading in files
uni_df = pd.read_csv('D:/Python/validere_takehome/CollegeScorecard_Raw_Data/combined_data/college_scorecard_fiveyears.csv', 
                     warn_bad_lines = False)
uni_df.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMENRUP_NOTFIRSTTIME_POOLED_SUPP,OMENRYP_FULLTIME_POOLED_SUPP,OMENRAP_FULLTIME_POOLED_SUPP,OMAWDP8_FULLTIME_POOLED_SUPP,OMENRUP_FULLTIME_POOLED_SUPP,OMENRYP_PARTTIME_POOLED_SUPP,OMENRAP_PARTTIME_POOLED_SUPP,OMAWDP8_PARTTIME_POOLED_SUPP,OMENRUP_PARTTIME_POOLED_SUPP,DATE
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MERGED2012_13_PP.csv
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MERGED2012_13_PP.csv
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MERGED2012_13_PP.csv
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MERGED2012_13_PP.csv
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MERGED2012_13_PP.csv


In [11]:
# what is the shape of the data?
uni_df.shape

(45180, 1978)

Essentially, each dataset corresponds to a specific school in a specific year. There are many features to the data with many features having mixed types! This will require a good amount of cleaning and reading of the data dictionary to drop unecessary features before conducting further analysis.

Looking at the data dictionary and sheer amount of features this dataset has, it would probably be best to pull useful features instead of dropping ones that aren't useful.

## Data cleaning

After a read-through of the technical documentation, I selected some features that I thought would be interesting to analyze and may be related to rising tuition. Broad categories these features fit under include the location/size of the university, tuition, revenue/expenses, faculty salaries, acceptance rates, SAT averages, average debt/grants, repayment period of debt after graduation, family earnings, and earnings after graduation.  

In [12]:
columns = ['INSTNM', 'CITY', 'STABBR', 'ZIP', 'NUMBRANCH', 'PREDDEG', 'HIGHDEG', 'LOCALE', 'LOCALE2',
                 'LATITUDE', 'LONGITUDE', 'CCBASIC', 'CCUGPROF', 'CCSIZSET', 'HBCU', 'PBI', 'NANTI', 'MENONLY',
                 'WOMENONLY', 'RELAFFIL', 'ADM_RATE', 'SAT_AVG', 'DISTANCEONLY', 'UG', 'CURROPER',
                 'NPT4_PUB' ,'NPT4_PRIV', 'NPT4_PROG', 'NPT4_OTHER', 'NUM4_PUB', 'NUM4_PRIV' ,'NUM4_PROG', 'NUM4_OTHER',
                 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'TUITFTE', 'INEXPFTE', 'AVGFACSAL', 'PFTFAC', 'PCTPELL', 'PCTFLOAN', 
                  'DEP_STAT_PCT_IND', 'DEP_INC_AVG', 'IND_INC_AVG', 'GRAD_DEBT_MDN', 'WDRAW_DEBT_MDN', 'MN_EARN_WNE_P6', 'DATE']

Time to split up the original dataframe into one that will actually be used for further analysis!

In [13]:
# splitting the dataframe
uni_df = uni_df[columns]
uni_df.shape

(45180, 48)

Only 48 columns to worry about, much better than looking at almost 2000.

I want to now look at what percentage of this data is missing, just to get a sense of the completeness of the data. I can get the percentage of non-missing values in each column by taking the result of the count method and dividing by the number of rows. 

In [15]:
uni_df.count() / len(uni_df)

INSTNM              1.000000
CITY                1.000000
STABBR              1.000000
ZIP                 1.000000
NUMBRANCH           1.000000
PREDDEG             1.000000
HIGHDEG             1.000000
LOCALE              0.147587
LOCALE2             0.000000
LATITUDE            0.147587
LONGITUDE           0.147587
CCBASIC             0.147587
CCUGPROF            0.147587
CCSIZSET            0.147587
HBCU                0.145795
PBI                 0.145795
NANTI               0.145795
MENONLY             0.147587
WOMENONLY           0.147587
RELAFFIL            0.020385
ADM_RATE            0.286853
SAT_AVG             0.178309
DISTANCEONLY        0.943382
UG                  0.000000
CURROPER            0.157415
NPT4_PUB            0.253608
NPT4_PRIV           0.592231
NPT4_PROG           0.000000
NPT4_OTHER          0.000000
NUM4_PUB            0.253918
NUM4_PRIV           0.595573
NUM4_PROG           0.000000
NUM4_OTHER          0.000000
TUITIONFEE_IN       0.569190
TUITIONFEE_OUT